In [1]:
import keras

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

ModuleNotFoundError: No module named 'google.colab'

In [3]:
#!pip install h5py
#conda install h5py --u

distributed 1.21.8 requires msgpack, which is not installed.
tensorflow 1.10.0 has requirement tensorboard<1.11.0,>=1.10.0, but you'll have tensorboard 1.12.2 which is incompatible.
keras 2.2.4 has requirement keras_applications>=1.0.6, but you'll have keras-applications 1.0.4 which is incompatible.
keras 2.2.4 has requirement keras_preprocessing>=1.0.5, but you'll have keras-preprocessing 1.0.2 which is incompatible.
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
filename = '/content/gdrive/My Drive/Files/SVHN_single_grey1.h5'
f = h5py.File(filename,'r')

OSError: Unable to open file (unable to open file: name = '/content/gdrive/My Drive/Files/SVHN_single_grey1.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [5]:
# List all groups
print("Keys: %s" % f.keys())
group_key = list(f.keys())
print(group_key)

NameError: name 'f' is not defined

In [6]:
x_train = np.array(f['X_train'])
y_train = np.array(f['y_train'])
x_val = np.array(f['X_val'])
y_val = np.array(f['y_val'])
x_test = np.array(f['X_test'])
y_test = np.array(f['y_test'])

NameError: name 'f' is not defined

In [7]:
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

NameError: name 'x_train' is not defined

In [8]:
import cv2

ModuleNotFoundError: No module named 'cv2'

In [9]:
def resize(data):
  return cv2.resize(data, dsize=(16,16), interpolation = cv2.INTER_LANCZOS4)

In [10]:
x_train_resize = np.zeros(shape =(42000,16,16))
x_test_resize = np.zeros(shape = (18000,16,16))
for i in range(0,len(x_train)):
  x_train_resize[i] = resize(x_train[i])
for i in range(0,len(x_test)):
  x_test_resize[i] = resize(x_test[i])  

NameError: name 'x_train' is not defined

In [11]:
x_train_reshape = x_train_resize.reshape(x_train_resize.shape[0],1024)
#x_train_reshape = x_train.reshape(x_train.shape[0],1024)
x_test_reshape = x_test_resize.reshape(x_test_resize.shape[0],1024)
#x_test_reshape = x_test.reshape(x_test.shape[0],1024)

ValueError: cannot reshape array of size 10752000 into shape (42000,1024)

In [12]:
scaler = StandardScaler()
eda_scaled = scaler.fit_transform(x_train_reshape)
eda_scaled_t = scaler.fit_transform(x_test_reshape)


NameError: name 'x_train_reshape' is not defined

def data_manipul(data):
  cov_matrix = np.cov(eda_scaled.T)
  e_vals, e_vecs = np.linalg.eig(cov_matrix)
  tot = sum(e_vals)
  var_exp = [( i /tot ) * 100 for i in sorted(e_vals, reverse=True)]
  cum_var_exp = np.cumsum(var_exp)
  print("Cumulative Variance Explained", cum_var_exp)
 -- plotting the chart......
  plt.figure(figsize=(10 , 5))
  plt.bar(range(1, e_vals.size + 1), var_exp, alpha = 0.5, align = 'center', label = 'Individual explained variance')
  plt.step(range(1, e_vals.size + 1), cum_var_exp, where='mid', label = 'Cumulative explained variance')
  plt.ylabel('Explained Variance Ratio')
  plt.xlabel('Principal Components')
  plt.legend(loc = 'best')
  plt.tight_layout()
  plt.show()

In [14]:
data_manipul(eda_scaled)

NameError: name 'eda_scaled' is not defined

In [15]:
from sklearn.decomposition import PCA
pca = PCA(n_components=45)
principalComponents = pca.fit_transform(eda_scaled)

NameError: name 'eda_scaled' is not defined

In [16]:
principalComponents_t = pca.transform(eda_scaled_t)

NameError: name 'eda_scaled_t' is not defined

In [17]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

In [18]:
accuracies = []
for i in range(30,100,10):
  model = KNeighborsClassifier(n_neighbors=i, weights ='distance', p=1)
  model.fit(principalComponents, y_train)
  # evaluate the model and update the accuracies list
  score = model.score(principalComponents_t, y_test)
  print("k=%d, accuracy=%.2f%%" % (i, score * 100))
  accuracies.append(score)

NameError: name 'principalComponents' is not defined

From above, we can see an accuracy of 61.89% for k=70 and on using distance based weighted KNN and Manhattan Distance as the metric.

In [19]:
model_opti = KNeighborsClassifier(n_neighbors=70, weights = 'distance', p=1, algorithm = 'ball_tree' )
model_opti.fit(principalComponents, y_train)
# evaluate the model and update the accuracies list
y_pred = model_opti.predict(principalComponents_t)

NameError: name 'principalComponents' is not defined

In [20]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))
score = model_opti.score(principalComponents_t, y_test)
print('Overall Accuracy Score', score)

NameError: name 'y_test' is not defined

In [21]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Dense, Activation, Dropout, Flatten, Reshape, BatchNormalization
from keras import backend as K
K.set_image_dim_ordering('th')

In [22]:
x_train = x_train.reshape(x_train.shape[0], 1,32,32).astype('float32')
x_test = x_test.reshape(x_test.shape[0], 1,32,32).astype('float32')
x_train /= 255
x_test /= 255

NameError: name 'x_train' is not defined

In [23]:
y_train_dum = keras.utils.to_categorical(y_train,num_classes=10)
y_test_dum = keras.utils.to_categorical(y_test,num_classes=10)

NameError: name 'y_train' is not defined

In [24]:
 from sklearn.metrics import classification_report
# Define Model
cnn = Sequential()
    
# 1st Conv Layer
cnn.add(Convolution2D(32,(3, 3), input_shape=(1,32,32), data_format='channels_first'))
cnn.add(BatchNormalization())
cnn.add(Activation('relu'))
# Dropout
cnn.add(Dropout(0.2))
# 2nd Conv Layer
cnn.add(Convolution2D(64,(3, 3), data_format='channels_first'))
cnn.add(BatchNormalization())
cnn.add(Activation('relu'))
# Max Pooling
cnn.add(MaxPooling2D(pool_size=(2,2)))
# Dropout
cnn.add(Dropout(0.2))
# 3rd Conv Layer
cnn.add(Convolution2D(64,(3, 3), data_format='channels_first'))
cnn.add(BatchNormalization())
cnn.add(Activation('relu'))
# Max Pooling
cnn.add(MaxPooling2D(pool_size=(2,2)))
# Dropout
cnn.add(Dropout(0.2))
# 4th Conv Layer
cnn.add(Convolution2D(64,(3, 3), data_format='channels_first'))
cnn.add(BatchNormalization())
cnn.add(Activation('relu'))
cnn.add(Dropout(0.25))
cnn.add(Flatten())
cnn.add(Dense(128, activation='relu'))
cnn.add(Dense(256, activation='relu'))
cnn.add(Dense(10, activation = 'softmax'))
cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=7, verbose=1, mode='auto')
callback_list = [early_stopping]

    # Train the model
    cnn.fit(x_train, y_train_dum, batch_size=32, nb_epoch=20, 
              validation_data=(x_test, y_test_dum), callbacks = callback_list)

IndentationError: unexpected indent (<ipython-input-24-f4fd6811a24a>, line 2)

In [25]:
y_pred = cnn.predict_classes(x_test)

#print(classification_report(y_test, y_pred))

NameError: name 'model_cnn' is not defined